In [5]:
import numpy as np
import pandas as pd
import gdreg
import os
import time
import matplotlib.pyplot as plt

# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
DATA_PATH='/n/groups/price/martin/WES_analysis/'
OUT_PATH = '/n/groups/price/martin/WES_analysis/toy_10K'

In [11]:
# trait_list
with open('/n/groups/price/martin/WES_analysis/toy_10K/trait_simu_debug/trait_list.txt', 'w') as f:
    for fname in sorted(os.listdir('/n/groups/price/martin/WES_analysis/toy_10K/trait_simu_debug')):
        if fname.endswith('.eff.gz'):
            f.write('%s\n' % fname.replace('.eff.gz',''))

### Read dic_data and annotations

In [3]:
# Check if the files are good
PGEN_FILE = OUT_PATH + '/chr@_v1.SPB.hg19.toy_10K'
dic_data = {}
for CHR in range(1,11):
    dic_data[CHR] = gdreg.util.read_pgen(PGEN_FILE.replace('@', '%s'%CHR))
    print(CHR, 'n_snp=%d'%dic_data[CHR]['pvar'].shape[0])
    if CHR==1:
        display(dic_data[CHR]['pvar'])
        display(dic_data[CHR]['psam'])
        display(dic_data[CHR]['afreq'])

1 n_snp=4232


,CHR,BP,SNP,REF,ALT
0,1,866511,1:931131:I:4.01,CCCCT,C
1,1,871334,1:935954:G:T,T,G
2,1,874816,1:939436:I:1,C,CT
3,1,876499,1:941119:A:G,G,A
4,1,877715,1:942335:C:G,G,C
...,...,...,...,...,...
4227,1,57415310,1:56949637:G:A,G,A
4228,1,57422511,1:56956838:C:T,C,T
4229,1,57425888,1:56960215:D:1,AG,A
4230,1,57477034,1:57011361:G:C,G,C


,FID,IID,SEX
0,3999004,3999004,2
1,4901875,4901875,2
2,5137618,5137618,1
3,5626389,5626389,2
4,1156626,1156626,2
...,...,...,...
9995,3249671,3249671,2
9996,1250387,1250387,2
9997,5824748,5824748,1
9998,4622825,4622825,2


,CHR,SNP,REF,ALT,MAF,OBS_CT
0,1,1:931131:I:4.01,CCCCT,C,0.267765,19364
1,1,1:935954:G:T,T,G,0.281250,20000
2,1,1:939436:I:1,C,CT,0.061506,19998
3,1,1:941119:A:G,G,A,0.052371,19992
4,1,1:942335:C:G,G,C,0.062123,19590
...,...,...,...,...,...,...
4227,1,1:56949637:G:A,G,A,0.049500,20000
4228,1,1:56956838:C:T,C,T,0.049600,20000
4229,1,1:56960215:D:1,AG,A,0.157488,19538
4230,1,1:57011361:G:C,G,C,0.110145,19892


2 n_snp=4056
3 n_snp=4067
4 n_snp=4027
5 n_snp=4106
6 n_snp=4154
7 n_snp=4071
8 n_snp=3891
9 n_snp=4149
10 n_snp=4129


In [4]:
# Read .annot files
dic_annot = {}
dic_annot['annot'] = gdreg.util.read_annot(OUT_PATH+'/toy.annot.gz')
display(dic_annot['annot'])
dic_annot['pannot'] = gdreg.util.read_annot(OUT_PATH+'/toy.pannot.gz')
display(dic_annot['pannot'])
dic_annot['pannot_hr'] = gdreg.util.read_annot(OUT_PATH+'/toy.pannot_hr.gz')
display(dic_annot['pannot_hr'])

,CHR,SNP,BP,CM,AN:CHR1t5_common,AN:CHR1t5_lf,AN:all_common,AN:all_lf
0,1,1:931131:I:4.01,866511,0,1,0,1,0
1,1,1:935954:G:T,871334,0,1,0,1,0
2,1,1:939436:I:1,874816,0,0,1,0,1
3,1,1:941119:A:G,876499,0,0,1,0,1
4,1,1:942335:C:G,877715,0,0,1,0,1
...,...,...,...,...,...,...,...,...
40877,10,10:116645109:A:G,118404620,0,0,0,1,0
40878,10,10:116664061:G:A,118423572,0,0,0,0,1
40879,10,10:116676557:G:T,118436068,0,0,0,1,0
40880,10,10:116692299:A:G,118451810,0,0,0,1,0


,CHR,SNP,BP,pAN:gene
0,1,1:931131:I:4.01,866511,GENE_1_0
1,1,1:935954:G:T,871334,GENE_1_0
2,1,1:939436:I:1,874816,GENE_1_0
3,1,1:941119:A:G,876499,GENE_1_0
4,1,1:942335:C:G,877715,GENE_1_0
...,...,...,...,...
40877,10,10:116645109:A:G,118404620,non-pAN
40878,10,10:116664061:G:A,118423572,non-pAN
40879,10,10:116676557:G:T,118436068,non-pAN
40880,10,10:116692299:A:G,118451810,non-pAN


,CHR,SNP,BP,pCHR,pSNP,pBP,pAN:proxy
0,1,1:931131:I:4.01,866511,1,1:935954:G:T,871334,1
1,1,1:931131:I:4.01,866511,1,1:939436:I:1,874816,1
2,1,1:931131:I:4.01,866511,1,1:941119:A:G,876499,1
3,1,1:935954:G:T,871334,1,1:939436:I:1,874816,1
4,1,1:935954:G:T,871334,1,1:941119:A:G,876499,1
...,...,...,...,...,...,...,...
121414,10,10:116638383:A:G,118397894,10,10:116645109:A:G,118404620,1
121415,10,10:116638460:A:G,118397971,10,10:116642054:C:T,118401565,1
121416,10,10:116638460:A:G,118397971,10,10:116645109:A:G,118404620,1
121417,10,10:116642054:C:T,118401565,10,10:116645109:A:G,118404620,1


### Simulate SNP effects and summarize 

In [13]:
# setup 
n_rep = 20
AN_list = ['AN:all_common', 'AN:all_lf', 'AN:CHR1t5_common', 'AN:CHR1t5_lf']
pAN_list = ['pAN:gene', 'pAN:proxy']

simu_list = [
    ('both_zero', [0, 0]),
    ('gene_pos', [0.1, 0]),
    ('gene_neg', [-0.1, 0]),
    ('proxy_pos', [0, 0.1]),
    ('proxy_neg', [0, -0.1]),
    ('both_pos', [0.1, 0.1]),
    ('both_neg', [-0.1, -0.1]),
    ('gene_pos_proxy_neg', [0.1, -0.1]),
    ('gene_neg_proxy_pos', [-0.1, 0.1]),
]

dic_simu = {}
for simu,param in simu_list:
    print(simu, param)
    dic_simu[simu] = {
        'AN:all_common' : 1,
        'AN:all_lf' :  1,
        'AN:CHR1t5_common' : -0.5,
        'AN:CHR1t5_lf' : 0.5,
        'pAN:gene' : param[0],
        'pAN:proxy' : param[1],
    }

both_zero [0, 0]
gene_pos [0.1, 0]
gene_neg [-0.1, 0]
proxy_pos [0, 0.1]
proxy_neg [0, -0.1]
both_pos [0.1, 0.1]
both_neg [-0.1, -0.1]
gene_pos_proxy_neg [0.1, -0.1]
gene_neg_proxy_pos [-0.1, 0.1]


In [14]:
# simulate
OUT_PATH = "/n/groups/price/martin/WES_analysis/toy_10K/trait_simu_debug"
df_annot = dic_annot['annot'].copy()
for simu in dic_simu:
    for i in range(n_rep):
        print(simu, i)
        df_effect = gdreg.simulate.simulate_snp_effect(
            df_annot, 
            dic_simu[simu], 
            pannot_list = [dic_annot['pannot']], 
            pannot_hr_list = [dic_annot['pannot_hr']],
            p_causal = 1,
            alpha=-1,
            verbose=False,
            random_seed=i,
        )

        df_summary = gdreg.simulate.summarize_snp_effect(
            df_effect,
            df_annot, 
            pannot_list = [dic_annot['pannot']],
            pannot_hr_list = [dic_annot['pannot_hr']],
            verbose=False
        )

        df_effect.to_csv(OUT_PATH+'/%s_rep%d.eff.gz'%(simu, i), sep='\t', index=False)
        df_summary.to_csv(OUT_PATH+'/%s_rep%d.eff_sum'%(simu, i), sep='\t', index=False)

both_zero 0


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


both_zero 1


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


both_zero 2


...

/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


gene_neg_proxy_pos 18


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


gene_neg_proxy_pos 19


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


### Old code

In [69]:
n_rep = 20
df_afreq = None
for CHR in dic_data:
    if df_afreq is None:
        df_afreq = dic_data[CHR]['afreq']
    else:
        df_afreq = pd.concat([df_afreq, dic_data[CHR]['afreq']], axis=0)

In [70]:
# Sanity_nd
dic_coef = {
    'AN:all_common' : 1,
    'AN:all_lf' : 1,
    'AN:CHR1t5_common' : -0.5,
    'AN:CHR1t5_lf' : 0.5,
    'pAN:gene' : 0,
    'pAN:proxy' : 0,
}

df_annot = dic_annot['annot'].copy()
df_annot['MAF'] = df_afreq['MAF'].values

AN_list = ['AN:all_common', 'AN:all_lf', 'AN:CHR1t5_common', 'AN:CHR1t5_lf']
pAN_list = ['pAN:gene', 'pAN:proxy']

dic_summary = {}

for i in range(n_rep):
    print(i)
    df_effect = gdreg.simulate.simulate_snp_effect(
        df_annot, 
        dic_coef, 
        pannot_list = [dic_annot['pannot']], 
        pannot_hr_list = [dic_annot['pannot_hr']],
        p_causal = 1,
        alpha=-1,
        verbose=False,
        random_seed=i,
    )

    dic_summary[i] = gdreg.simulate.summarize_snp_effect(
        df_effect,
        df_annot, 
        pannot_list = [dic_annot['pannot']],
        pannot_hr_list = [dic_annot['pannot_hr']],
        verbose=False
    )
    
    df_effect.to_csv(OUT_PATH+'/sanity_nd_rep%d.eff.gz'%i, sep='\t', index=False)
    dic_summary[i].to_csv(OUT_PATH+'/sanity_nd_rep%d.eff_sum'%i, sep='\t', index=False)
    
# Summary 
temp_dic = dic_summary.copy()
for AN in AN_list:
    true_val = dic_coef[AN] if AN in dic_coef else 0
    v_tau = [temp_dic[x].loc[AN, 'coef'] for x in range(n_rep)]
    tau_mean, tau_sd = np.mean(v_tau), np.std(v_tau)
    v_h2_ps = [temp_dic[x].loc[AN, 'aggeff'] / temp_dic[x].loc[AN, 'size'] for x in range(n_rep)]
    h2_ps_mean, h2_ps_sd = np.mean(v_h2_ps), np.std(v_h2_ps)
    print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s (true=%0.2g)'%(AN, true_val), 
            'tau=%0.2g (SD=%0.2g)' % (tau_mean, tau_sd),
            'h2_ps=%0.2g (SD=%0.2g)' % (h2_ps_mean, h2_ps_sd),
        ))
    
for pAN in pAN_list:
    true_val = dic_coef[pAN] if pAN in dic_coef else 0
    v_rho = [temp_dic[x].loc[pAN, 'coef'] for x in range(n_rep)]
    rho_mean, rho_sd = np.mean(v_rho), np.std(v_rho)
    v_r2_ps = [temp_dic[x].loc[pAN, 'aggeff'] / temp_dic[x].loc[AN, 'size'] for x in range(n_rep)]
    r2_ps_mean, r2_ps_sd = np.mean(v_rho), np.std(v_rho)
    print('{:^30s}|{:^30s}|{:^30s}'.format(
        '%s|%s (true=%0.2g)'%(pAN,AN,true_val), 
        'rho=%0.2g (SD=%0.2g)' % (rho_mean, rho_sd),
        'cov_ps=%0.2g (SD=%0.2g)' % (r2_ps_mean, r2_ps_sd),
    ))

0


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


1


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


2


...

18


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


19


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


    AN:all_common (true=1)    |    tau=1.2e-05 (SD=3e-07)    |   h2_ps=9.1e-06 (SD=2e-07)   
      AN:all_lf (true=1)      |   tau=1.2e-05 (SD=2.8e-07)   |  h2_ps=1.5e-05 (SD=4.4e-07)  
 AN:CHR1t5_common (true=-0.5) |  tau=-6.1e-06 (SD=2.8e-07)   |  h2_ps=6.1e-06 (SD=1.5e-07)  
   AN:CHR1t5_lf (true=0.5)    |    tau=6e-06 (SD=5.1e-07)    |  h2_ps=1.8e-05 (SD=6.7e-07)  
pAN:gene|AN:CHR1t5_lf (true=0)|    rho=2.2e-09 (SD=1e-07)    |  cov_ps=2.2e-09 (SD=1e-07)   
pAN:proxy|AN:CHR1t5_lf (true=0)|  rho=-1.9e-08 (SD=4.9e-08)   | cov_ps=-1.9e-08 (SD=4.9e-08) 


In [71]:
# sanity
dic_coef = {
    'AN:all_common' : 1,
    'AN:all_lf' : 1,
    'AN:CHR1t5_common' : -0.5,
    'AN:CHR1t5_lf' : 0.5,
    'pAN:gene' : 0.2,
    'pAN:proxy' : -0.2,
}

df_annot = dic_annot['annot'].copy()
df_annot['MAF'] = df_afreq['MAF'].values

AN_list = ['AN:all_common', 'AN:all_lf', 'AN:CHR1t5_common', 'AN:CHR1t5_lf']
pAN_list = ['pAN:gene', 'pAN:proxy']

dic_summary = {}

for i in range(n_rep):
    print(i)
    df_effect = gdreg.simulate.simulate_snp_effect(
        df_annot, 
        dic_coef, 
        pannot_list = [dic_annot['pannot']], 
        pannot_hr_list = [dic_annot['pannot_hr']],
        p_causal = 1,
        alpha=-1,
        verbose=False,
        random_seed=i,
    )

    dic_summary[i] = gdreg.simulate.summarize_snp_effect(
        df_effect,
        df_annot, 
        pannot_list = [dic_annot['pannot']],
        pannot_hr_list = [dic_annot['pannot_hr']],
        verbose=False
    )
    
    df_effect.to_csv(OUT_PATH+'/sanity_rep%d.eff.gz'%i, sep='\t', index=False)
    dic_summary[i].to_csv(OUT_PATH+'/sanity_rep%d.eff_sum'%i, sep='\t', index=False)
    
# Summary 
temp_dic = dic_summary.copy()
for AN in AN_list:
    true_val = dic_coef[AN] if AN in dic_coef else 0
    v_tau = [temp_dic[x].loc[AN, 'coef'] for x in range(n_rep)]
    tau_mean, tau_sd = np.mean(v_tau), np.std(v_tau)
    v_h2_ps = [temp_dic[x].loc[AN, 'aggeff'] / temp_dic[x].loc[AN, 'size'] for x in range(n_rep)]
    h2_ps_mean, h2_ps_sd = np.mean(v_h2_ps), np.std(v_h2_ps)
    print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s (true=%0.2g)'%(AN, true_val), 
            'tau=%0.2g (SD=%0.2g)' % (tau_mean, tau_sd),
            'h2_ps=%0.2g (SD=%0.2g)' % (h2_ps_mean, h2_ps_sd),
        ))
    
for pAN in pAN_list:
    true_val = dic_coef[pAN] if pAN in dic_coef else 0
    v_rho = [temp_dic[x].loc[pAN, 'coef'] for x in range(n_rep)]
    rho_mean, rho_sd = np.mean(v_rho), np.std(v_rho)
    v_r2_ps = [temp_dic[x].loc[pAN, 'aggeff'] / temp_dic[x].loc[AN, 'size'] for x in range(n_rep)]
    r2_ps_mean, r2_ps_sd = np.mean(v_rho), np.std(v_rho)
    print('{:^30s}|{:^30s}|{:^30s}'.format(
        '%s|%s (true=%0.2g)'%(pAN,AN,true_val), 
        'rho=%0.2g (SD=%0.2g)' % (rho_mean, rho_sd),
        'cov_ps=%0.2g (SD=%0.2g)' % (r2_ps_mean, r2_ps_sd),
    ))

0


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


1


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


2


...

18


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


19


/home/jz286/myenv/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


    AN:all_common (true=1)    |   tau=1.3e-05 (SD=3.6e-07)   |  h2_ps=9.4e-06 (SD=2.2e-07)  
      AN:all_lf (true=1)      |   tau=1.2e-05 (SD=2.9e-07)   |  h2_ps=1.6e-05 (SD=4.4e-07)  
 AN:CHR1t5_common (true=-0.5) |  tau=-6.2e-06 (SD=3.3e-07)   |  h2_ps=6.3e-06 (SD=1.6e-07)  
   AN:CHR1t5_lf (true=0.5)    |   tau=6.2e-06 (SD=4.6e-07)   |  h2_ps=1.9e-05 (SD=6.4e-07)  
pAN:gene|AN:CHR1t5_lf (true=0.2)|    rho=2.3e-06 (SD=5e-07)    |  cov_ps=2.3e-06 (SD=5e-07)   
pAN:proxy|AN:CHR1t5_lf (true=-0.2)|  rho=-1.5e-06 (SD=4.2e-08)   | cov_ps=-1.5e-06 (SD=4.2e-08) 


### Old code

In [8]:
df_afreq = None
for CHR in dic_data:
    if df_afreq is None:
        df_afreq = dic_data[CHR]['afreq']
    else:
        df_afreq = pd.concat([df_afreq, dic_data[CHR]['afreq']], axis=0)

dic_coef = {
    'AN:ALL' : 1,
    'AN:CHR1t5' : -0.5,
    'AN:ODD' : 0.5,
    'pAN:gene|AN:ALL' : 0.2,
    'pAN:proxy|AN:ALL' : -0.2,
}

df_annot = dic_annot['annot'].copy()
df_annot['MAF'] = df_afreq['MAF'].values

AN_list = ['AN:ALL', 'AN:CHR1t5', 'AN:ODD']
pAN_list = ['pAN:gene', 'pAN:proxy']

In [9]:
# Sanity
n_rep = 20 
dic_summary = {}

for i in range(n_rep):
    print(i)
    df_effect = gdreg.simulate.simulate_snp_effect(
        df_annot, 
        dic_coef, 
        pannot_list = [dic_annot['pannot']], 
        pannot_hr_list = [dic_annot['pannot_hr']],
        p_causal = 1,
        alpha=-1,
        verbose=False,
        random_seed=i,
    )

    dic_summary[i] = gdreg.simulate.summarize_snp_effect(
        df_effect,
        df_annot, 
        pannot_list = [dic_annot['pannot']],
        pannot_hr_list = [dic_annot['pannot_hr']],
        verbose=False
    )
    
    df_effect.to_csv(OUT_PATH+'/sanity_rep%d.eff.gz'%i, sep='\t', index=False)
    dic_summary[i].to_csv(OUT_PATH+'/sanity_rep%d.eff_sum'%i, sep='\t', index=False)
    
# Summary 
temp_dic = dic_summary.copy()
for AN in AN_list:
    true_val = dic_coef[AN] if AN in dic_coef else 0
    v_tau = [temp_dic[x].loc[AN, 'tau'] for x in range(n_rep)]
    tau_mean, tau_sd = np.mean(v_tau), np.std(v_tau)
    v_h2_ps = [temp_dic[x].loc[AN, 'h2_ps'] for x in range(n_rep)]
    h2_ps_mean, h2_ps_sd = np.mean(v_h2_ps), np.std(v_h2_ps)
    print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s (true=%0.2g)'%(AN, true_val), 
            'tau=%0.2g (SD=%0.2g)' % (tau_mean, tau_sd),
            'h2_ps=%0.2g (SD=%0.2g)' % (h2_ps_mean, h2_ps_sd),
        ))
    
for AN in AN_list:
    for pAN in pAN_list:
        true_val = dic_coef['%s|%s'%(pAN,AN)] if '%s|%s'%(pAN,AN) in dic_coef else 0
        v_rho = [temp_dic[x].loc[AN, 'rho|%s'%pAN] for x in range(n_rep)]
        rho_mean, rho_sd = np.mean(v_rho), np.std(v_rho)
        v_r2_ps = [temp_dic[x].loc[AN, 'r2_ps|%s'%pAN] for x in range(n_rep)]
        r2_ps_mean, r2_ps_sd = np.mean(v_rho), np.std(v_rho)
        print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s|%s (true=%0.2g)'%(pAN,AN,true_val), 
            'rho=%0.2g (SD=%0.2g)' % (rho_mean, rho_sd),
            'r2_ps=%0.2g (SD=%0.2g)' % (r2_ps_mean, r2_ps_sd),
        ))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
       AN:ALL (true=1)        |   tau=1.2e-05 (SD=2.9e-07)   |   h2_ps=1.3e-05 (SD=3e-07)   
    AN:CHR1t5 (true=-0.5)     |  tau=-6.2e-06 (SD=1.5e-07)   |   h2_ps=1e-05 (SD=2.5e-07)   
      AN:ODD (true=0.5)       |   tau=6.3e-06 (SD=1.7e-07)   |  h2_ps=1.5e-05 (SD=3.7e-07)  
  pAN:gene|AN:ALL (true=0.2)  |     rho=0.18 (SD=0.066)      |    r2_ps=0.18 (SD=0.066)     
 pAN:proxy|AN:ALL (true=-0.2) |    rho=-0.13 (SD=0.0023)     |   r2_ps=-0.13 (SD=0.0023)    
 pAN:gene|AN:CHR1t5 (true=0)  |    rho=-0.018 (SD=0.042)     |   r2_ps=-0.018 (SD=0.042)    
 pAN:proxy|AN:CHR1t5 (true=0) |   rho=-0.0014 (SD=0.0031)    |  r2_ps=-0.0014 (SD=0.0031)   
   pAN:gene|AN:ODD (true=0)   |    rho=0.0085 (SD=0.093)     |   r2_ps=0.0085 (SD=0.093)    
  pAN:proxy|AN:ODD (true=0)   |    rho=0.019 (SD=0.0023)     |   r2_ps=0.019 (SD=0.0023)    


In [25]:
# # Realistic
# n_rep = 10 
# dic_summary = {}

# for i in range(n_rep):
#     print(i)
#     df_effect = gdreg.simulate.simulate_snp_effect(
#         df_annot, 
#         dic_coef, 
#         pannot_list = [dic_annot['pannot']], 
#         pannot_hr_list = [dic_annot['pannot_hr']],
#         verbose=False,
#         random_seed=i,
#     )

#     dic_summary[i] = gdreg.simulate.summarize_snp_effect(
#         df_effect,
#         df_annot, 
#         pannot_list = [dic_annot['pannot']],
#         pannot_hr_list = [dic_annot['pannot_hr']],
#         verbose=False
#     )
    
# # Summary 
# temp_dic = dic_summary.copy()
# for AN in AN_list:
#     true_val = dic_coef[AN] if AN in dic_coef else 0
#     v_tau = [temp_dic[x].loc[AN, 'tau'] for x in range(n_rep)]
#     tau_mean, tau_sd = np.mean(v_tau), np.std(v_tau)
#     v_h2_ps = [temp_dic[x].loc[AN, 'h2_ps'] for x in range(n_rep)]
#     h2_ps_mean, h2_ps_sd = np.mean(v_h2_ps), np.std(v_h2_ps)
#     print('{:^30s}|{:^30s}|{:^30s}'.format(
#             '%s (true=%0.2g)'%(AN, true_val), 
#             'tau=%0.2g (SD=%0.2g)' % (tau_mean, tau_sd),
#             'h2_ps=%0.2g (SD=%0.2g)' % (h2_ps_mean, h2_ps_sd),
#         ))
    
# for AN in AN_list:
#     for pAN in pAN_list:
#         true_val = dic_coef['%s|%s'%(pAN,AN)] if '%s|%s'%(pAN,AN) in dic_coef else 0
#         v_rho = [temp_dic[x].loc[AN, 'rho|%s'%pAN] for x in range(n_rep)]
#         rho_mean, rho_sd = np.mean(v_rho), np.std(v_rho)
#         v_r2_ps = [temp_dic[x].loc[AN, 'r2_ps|%s'%pAN] for x in range(n_rep)]
#         r2_ps_mean, r2_ps_sd = np.mean(v_rho), np.std(v_rho)
#         print('{:^30s}|{:^30s}|{:^30s}'.format(
#             '%s|%s (true=%0.2g)'%(pAN,AN,true_val), 
#             'rho=%0.2g (SD=%0.2g)' % (rho_mean, rho_sd),
#             'r2_ps=%0.2g (SD=%0.2g)' % (r2_ps_mean, r2_ps_sd),
#         ))

0
1
2
3
4
5
6
7
8
9
       AN:ALL (true=1)        |   tau=2.8e-05 (SD=2.1e-06)   |  h2_ps=3.7e-05 (SD=2.4e-06)  
       AN:CHR1 (true=1)       |   tau=2.6e-05 (SD=7.5e-06)   |  h2_ps=6.6e-05 (SD=6.4e-06)  
      AN:ODD (true=0.5)       |   tau=1.3e-05 (SD=1.6e-06)   |  h2_ps=4.6e-05 (SD=2.9e-06)  
   pAN:gene|AN:ALL (true=0)   |    rho=0.0014 (SD=0.0043)    |   r2_ps=0.0014 (SD=0.0043)   
  pAN:proxy|AN:ALL (true=0)   |   rho=-0.00092 (SD=0.0092)   |  r2_ps=-0.00092 (SD=0.0092)  
 pAN:gene|AN:CHR1 (true=0.8)  |     rho=0.14 (SD=0.091)      |    r2_ps=0.14 (SD=0.091)     
pAN:proxy|AN:CHR1 (true=-0.2) |    rho=-0.033 (SD=0.013)     |   r2_ps=-0.033 (SD=0.013)    
   pAN:gene|AN:ODD (true=0)   |    rho=-0.003 (SD=0.0066)    |   r2_ps=-0.003 (SD=0.0066)   
  pAN:proxy|AN:ODD (true=0)   |   rho=-0.0022 (SD=0.0099)    |  r2_ps=-0.0022 (SD=0.0099)   
